## Document Loading

Load a blog post on agents.

In [10]:
! pip install pdf2image

  Using cached pdf2image-1.16.3-py3-none-any.whl (11 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


Academic papers.

In [7]:
import arxiv
from langchain.chat_models import ChatAnthropic
from langchain.document_loaders import ArxivLoader, UnstructuredPDFLoader

# Load a paper to use
paper = next(arxiv.Search(query="Visual Instruction Tuning").results())
paper.download_pdf(filename="downloaded-paper.pdf")
loader = UnstructuredPDFLoader("downloaded-paper.pdf")
doc = loader.load()[0]

Also try loading blog posts.

In [3]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
text = loader.load()

## Connect to template

`Context`
 
* LangServe apps gives you access to templates.
* Templates LLM pipeline (runnables or chains) end-points accessible via FastAPI.
* The environment for these templates is managed by Poetry.

`Create app`

* Install LangServe and create an app.
* This will create a new Poetry environment /
```
pip install < to add > 
langchain serve new my-app
cd my-app
```

`Add templates`

* When we add a template, we update the Poetry config file with the necessary dependencies.
* It also automatically installed these template dependencies in your Poetry environment
```
langchain serve add summarize-anthropic
```

`Start FastAPI server`

```
langchain start
```

Note, we can now look at the endpoints:

http://127.0.0.1:8000/docs#

And look specifically at our loaded template:

http://127.0.0.1:8000/docs#/default/invoke_summarize_anthropic_invoke_post
 
We can also use remote runnable to call it.

## Summarization

We will use [this](https://smith.langchain.com/hub/hwchase17/anthropic-paper-qa) prompt.

In [15]:
from langserve.client import RemoteRunnable

summarization = RemoteRunnable("http://localhost:8000/summarize-anthropic")
summarization.invoke({"text": doc.page_content})

" Here is my attempt to complete the requested tasks:\n\n<kindergarten_abstract>\nThe scientists made a robot friend that can see pictures and talk about them. First they let a big talking robot make up fun conversations about pictures. Then they taught their robot friend to have conversations too. Their robot friend got pretty good at seeing pictures and talking about them!\n</kindergarten_abstract>\n\n<moosewood_methods>\nVisual Instruction Tuning\nRecipe adapted from Haotian Liu et al.\n\nIngredients:\n- 1 large language model (we used GPT-4)\n- 158,000 images with text descriptions \n- 1 vision model (we used CLIP)\n- 1 large multimodal model (we used LLaMA)\n\nInstructions:\n1. Have the GPT-4 model make up conversations, long descriptions, and complex stories about the images, using only the text descriptions as hints. \n2. Connect the vision model and language model together into one big multimodal model. This will be your robot friend!\n3. Show the robot friend examples of the c